In [1]:
import json
import os
import random
import re
import time

In [2]:
from bs4 import BeautifulSoup
import bs4
import cloudscraper
import geopandas as gpd
import pandas as pd

In [3]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [4]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [5]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Population data

In [6]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [7]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [8]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get KFC Data

In [9]:
scraper = cloudscraper.create_scraper()

In [10]:
def restaurant_open(div_teaser_open: bs4.element.Tag) -> bool:
    open_list = []
    for span in div_teaser_open.find_all("span")[0].find_all("span"):
        if span.text:
            open_list.append(span.text.strip())
    times_open = list(set(open_list))
    if len(times_open) == 1 and times_open[0] == "Closed Today":
        return False
    return True

In [11]:
def get_coords(url: str) -> Point:
    r = scraper.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    latitude_meta = soup.find("meta", {"itemprop": "latitude"})
    longitude_meta = soup.find("meta", {"itemprop": "longitude"})
    lat = float(latitude_meta.attrs["content"])
    lon = float(longitude_meta.attrs["content"])
    return Point(lon, lat)

In [12]:
store_list = []
url_base = "https://locations.kfc.com/"
for i in tqdm(range(len(states_df)), desc="Parsing States"):
    # Get State code
    state_store_list = []
    state_code = states_df.iloc[i]["STUSPS"].lower()

    # If File exists continue
    if os.path.isfile(f"data/states/{state_code}.gpkg"):
        continue

    # Get all locations in a state
    url = os.path.join(url_base, state_code)
    r = scraper.get(url)

    if r.status_code != 200:
        print(states_df.iloc[i]["NAME"])
        continue

    soup = BeautifulSoup(r.text, "html.parser")
    locations_as = soup.find_all("a", {"class": "Directory-listLink"})

    # Parse all locations in a state
    closed = 0
    for locations_a in tqdm(locations_as, desc=f"Parsing Locations In {state_code}"):
        href = locations_a.attrs["href"]
        count = int(re.findall(r"\d+", locations_a.attrs["data-count"])[0])
        location_url = url_base + href

        if count == 1:
            point = get_coords(location_url)
            store_dict = {
                "STATE": state_code,
                "geometry": point,
            }
            store_list.append(store_dict)
            state_store_list.append(store_dict)
        else:
            r_location = scraper.get(location_url)
            location_soup = BeautifulSoup(r_location.text, "html.parser")
            locations_lis = location_soup.find_all(
                "li", {"class": "Directory-listTeaser"}
            )

            # Multiple stores in locations, parse the stores per location
            for locations_li in locations_lis:
                try:
                    # Get url before open so I can print it to check if it's actually closed
                    location_a = locations_li.find("a", {"class": "Teaser-titleLink"})
                    href = location_a.attrs["href"]
                    location_url = os.path.join(url_base, href.replace("../", ""))

                    # Determine if open
                    div_teaser_open = locations_li.find("div", {"class": "Teaser-open"})
                    restaurant_open_b = restaurant_open(div_teaser_open)
                    if not restaurant_open_b:
                        print(location_url, "Closed")
                        closed += 1
                        continue

                    point = get_coords(location_url)
                    store_dict = {
                        "STATE": state_code,
                        "geometry": point,
                    }
                    store_list.append(store_dict)
                    state_store_list.append(store_dict)
                except Exception as e:
                    print(e, location_url)
                time.sleep(random.uniform(0.01, 0.5))

    print(state_code, f"Closed: {closed}")
    if state_store_list:
        kfc_state_gdf = gpd.GeoDataFrame(state_store_list, crs=4326)
        kfc_state_gdf.to_file(f"data/states/{state_code}.gpkg")

Parsing States:   0%|          | 0/56 [00:00<?, ?it/s]

Parsing Locations In ms:   0%|          | 0/57 [00:00<?, ?it/s]

https://locations.kfc.com/ms/brookhaven/755-brookway-boulevard Closed
https://locations.kfc.com/ms/laurel/724-chantilly Closed
https://locations.kfc.com/ms/vicksburg/3144-indiana-avenue Closed
ms Closed: 3


Parsing Locations In nc:   0%|          | 0/121 [00:00<?, ?it/s]

https://locations.kfc.com/nc/belmont/6813-e-wilkinson-boulevard-C426002 Closed
https://locations.kfc.com/nc/boiling-springs/219-south-main-st-C426007 Closed
https://locations.kfc.com/nc/charlotte/4601-n-tryon-st Closed
https://locations.kfc.com/nc/charlotte/8800-pineville-matthews-rd Closed
https://locations.kfc.com/nc/fayetteville/3002-raeford-road Closed
https://locations.kfc.com/nc/forest-city/131-bethany-church-road-C426003 Closed
https://locations.kfc.com/nc/greenville/2000-south-east-greenville-boulevard Closed
https://locations.kfc.com/nc/henderson/130-raleigh-road Closed
https://locations.kfc.com/nc/kings-mountain/409-e-king-street-C426001 Closed
https://locations.kfc.com/nc/marion/1110-highway-70-w Closed
https://locations.kfc.com/nc/mooresville/429-e-plaza-drive-C426004 Closed
https://locations.kfc.com/nc/morganton/324-carbon-city-rd Closed
https://locations.kfc.com/nc/rutherfordton/115-college-ave-C426005 Closed
https://locations.kfc.com/nc/siler-city/1702-east-11th-street-7

...

https://locations.kfc.com/in/carmel/10575-n-michigan-road Closed
https://locations.kfc.com/in/evansville/1201-covert-avenue Closed
https://locations.kfc.com/in/indianapolis/7215-n-michigan-road Closed
'NoneType' object has no attribute 'find_all' https://locations.kfc.com/in/indianapolis/8456-michigan-rd-H740024_D
https://locations.kfc.com/in/jeffersonville/1426-east-tenth-street Closed
https://locations.kfc.com/in/ligonier/241-us-highway-6-s-37838 Closed
https://locations.kfc.com/in/ligonier/241-us-highway-6-s-C119015 Closed
in Closed: 6


Parsing Locations In mn:   0%|          | 0/40 [00:00<?, ?it/s]

mn Closed: 0


Parsing Locations In ct:   0%|          | 0/33 [00:00<?, ?it/s]

https://locations.kfc.com/ct/norwalk/75-richards-ave-F033001 Closed
ct Closed: 1


#### If the above cells breaks just keep running until complete. Then run the line below

In [13]:
kfc_state_gdfs = []
states_path = "data/states"
for file in os.listdir("data/states"):
    constructed_path = os.path.join(states_path, file)
    if constructed_path.endswith(".gpkg"):
        state_kfc_gdf = gpd.read_file(constructed_path)
        kfc_state_gdfs.append(state_kfc_gdf)

In [14]:
kfc_state_gdf = gpd.GeoDataFrame(pd.concat(kfc_state_gdfs, ignore_index=True))

In [15]:
kfc_state_gdf = kfc_state_gdf.to_crs(9311)
kfc_state_gdf.to_file(f"data/stores.gpkg")

## Combine With States

In [16]:
kfc_state_counts_series = kfc_state_gdf.groupby("STATE").size()
kfc_state_counts_df = pd.DataFrame(
    {
        "STUSPS": kfc_state_counts_series.index,
        "stores": kfc_state_counts_series.values,
    }
)

In [17]:
kfc_state_counts_df["STUSPS"] = kfc_state_counts_df["STUSPS"].str.upper()

In [18]:
kfc_state_counts_df = states_with_population_df.merge(
    kfc_state_counts_df, on="STUSPS", how="left"
)

In [19]:
kfc_state_counts_df = kfc_state_counts_df.fillna(0)
kfc_state_counts_df["stores"] = kfc_state_counts_df["stores"].astype(int)

In [20]:
kfc_state_counts_df["per_1000"] = kfc_state_counts_df["stores"] / (
    kfc_state_counts_df["POPULATION"] / 1000
)
kfc_state_counts_df["per_10k"] = kfc_state_counts_df["stores"] / (
    kfc_state_counts_df["POPULATION"] / 10_000
)
kfc_state_counts_df["per_100k"] = kfc_state_counts_df["stores"] / (
    kfc_state_counts_df["POPULATION"] / 100000
)
kfc_state_counts_df["per_500k"] = kfc_state_counts_df["stores"] / (
    kfc_state_counts_df["POPULATION"] / 500_000
)
kfc_state_counts_df["per_1m"] = kfc_state_counts_df["stores"] / (
    kfc_state_counts_df["POPULATION"] / 1_000_000
)

In [21]:
kfc_state_counts_df = kfc_state_counts_df.dropna()

In [22]:
kfc_state_counts_df = kfc_state_counts_df.to_crs(9311)
kfc_state_counts_df.to_file(f"data/stores_by_states.gpkg")